# Árboles de decisión

Un **árbol de decisión** es una estructura de árbol similar a un diagrama de flujo donde un nodo interno representa una característica (o atributo), la rama representa una regla de decisión y cada nodo de hoja representa el resultado. El nodo superior de un **árbol de decisión** se conoce como nodo raíz. Aprende a particionar en base al valor del atributo. Particiona el árbol de manera recursiva llamada partición recursiva.

El **árbol de decisión** es un algoritmo de ML de tipo caja blanca. Comparte la lógica de toma de decisiones interna, a diferencia de los algoritmos de caja negra como las redes neuronales. Esto quiere decir que los árboles de decisión son **modelos explicativos**.

Su complejidad temporal de entrenamiento es menor en comparación con otros algoritmos clásicos de clasificación. La complejidad temporal de los árboles de decisión depende del número de registros y atributos de los datos dados. El árbol de decisión es un método no paramétrico (también conocido como libre de distribución), por lo que no presuponen una determinada distribución en los datos de entrada para poder funcionar.


## Pseudo-código del algoritmo de árbol de decisión
La idea básica detrás de cualquier algoritmo de árbol de decisión es la siguiente:

1. Seleccionar el mejor atributo utilizando Medidas de selección de atributos (ASM, Attribute Selection Metric en inglés) para dividir los registros.
2. Hacer de ese atributo un nodo de decisión y dividir el conjunto de datos en subconjuntos más pequeños.
3. Comenzar a construir árboles repitiendo este proceso recursivamente para cada nodo hijo hasta que una de las siguientes condiciones se cumpla:

    * Todas las observaciones (filas) se corresponden con el mismo valor de atributo.
    * No quedan más atributos
    * No quedan más filas


## Medidas de selección de atributos
Una medida de selección de atributos es una heurística para seleccionar el criterio de división que divide los datos de la mejor manera posible. También se conoce como reglas de **partición** porque nos ayuda a determinar puntos de ruptura en un nodo dado. Estas medidas proporcionan una puntuación a cada característica (o atributo). El atributo de mejor puntuación se seleccionará como atributo de división.

Las medidas de selección más populares son la ganancia de información (_Information Gain_), y el coeficiente de Gini (_Gini Impurity_).


### Information Gain
[Shannon](https://es.wikipedia.org/wiki/Claude_Elwood_Shannon) inventó el concepto de [entropía](https://es.wikipedia.org/wiki/Entrop%C3%ADa_(informaci%C3%B3n)), que mide la impureza del conjunto de entradas. En física y matemáticas, la entropía se refiere a la aleatoriedad o la impureza en el sistema. En la teoría de la información, se refiere a la impureza en un grupo de muestras. 

La **ganancia de información** es la disminución de la entropía. La ganancia de información calcula la diferencia entre la entropía *antes de la división* y la entropía media *después de la división* del conjunto de datos basada en los valores de atributo dados.

La formulación matemática de esta métrica es la siguiente:

$\begin{eqnarray}
IG(T,a) &=& H(T) - H(T|a) \\
H(T) &=& -\sum_{i\in C}^{}p_i \log_2(p_i) \\
H(T|a) &=& \sum_{j\in vals(a)}^{}\frac{\left | T_j \right |}{\left | T \right |} H(T_j)
\end{eqnarray}$

donde $C$ es el conjunto de clases o etiquetas de nuestro conjunto de datos $T$, $p_i$ es la probabilidad de que una observación (fila) del conjunto de datos pertenezca a la clase $i$, $a$ es una característica (columna) del conjunto de datos y $vals(a)$ son todos los valores que toma la característica $a$.

### Coeficiente de Gini
La impureza de Gini se mide como:

$\begin{eqnarray}Gini(T) = 1 - \sum_{i\in C}^{}p_i^2\end{eqnarray}$

El **coeficiente de Gini** considera una división binaria para cada atributo. Se puede calcular una suma ponderada de la impureza de cada partición. Si una división binaria en las particiones del atributo $a$ divide los datos $T$ en $T_1$ y $T_2$, el coeficiente de Gini de $T$ es:

$\begin{eqnarray}Gini(T|a) = \frac{\left | T_1 \right |}{\left | T \right |}Gini(T_1)+\frac{\left | T_2 \right |}{\left | T \right |}Gini(T_2)\end{eqnarray}$

En el caso de un atributo discreto, el subconjunto que da el coeficiente de Gini más pequeño para el atributo elegido se selecciona como atributo de división. En el caso de los atributos continuos, la estrategia es seleccionar cada par de valores adyacentes como un posible punto de división y el punto con el coeficiente de Gini más pequeño es elegido como punto de división.

El atributo con el coeficiente mínimo de Gini se elige como atributo de división.


## Caso de estudio: clasificación de vinos con árboles de decisión
Vamos a usar el conjunto de datos incluido en `sklearn` sobre [clasificación de vinos](https://scikit-learn.org/stable/datasets/index.html#wine-dataset) para probar los árboles de decisión.

En primer lugar se importan las librerías necesarias:

In [0]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.datasets import load_wine
from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
import pydotplus

Cargamos el _dataset_

In [0]:
data = load_wine()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.DataFrame(data.target, columns=['Class'])
X.tail()

Hagamos una división rápida del conjunto de datos en entrenamiento y test:

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1337)

Entrenamos el modelo con el conjunto de entrenamiento:

In [0]:
clf = DecisionTreeClassifier(criterion='entropy', splitter='random', max_depth=3)
clf = clf.fit(X_train,y_train)

Y evaluamos el _accuracy_ del árbol de decisión con el conjunto de test:

In [0]:
print("Accuracy:",metrics.accuracy_score(y_test, clf.predict(X_test)))
print("F1:",metrics.f1_score(y_test, clf.predict(X_test), average='weighted'))

Gracias a que los árboles de decisión son un modelo de caja blanca, podemos obtener información muy valiosa sobre cómo realiza las particiones y lo que ello implica.

Vamos a representar gráficamente el árbol de decisión. Para ello convertimos el árbol de decisión en un grafo y lo mostramos por pantalla:

In [0]:
#plt.rcParams["figure.figsize"] = [20, 20]

dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = data.feature_names,class_names=data.target_names)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

## Caso de estudio: dibujando las fronteras de decisión de un árbol

Para ayudar a entender cómo funciona un **árbol de decisión** vamos a generar un conjunto de datos sintético con `make_blobs`. Generamos un *dataset* sintético con 3 clases y 1000 observaciones:

In [0]:
from sklearn.datasets import make_blobs
from matplotlib import pyplot as plt

X,y = make_blobs(1000, 2, 3, cluster_std=3, random_state=17)

plt.figure(figsize=(10, 5))
plt.scatter(X[y == 0, 0], X[y == 0, 1], 
            c='red', s=40, label='0')
plt.scatter(X[y == 1, 0], X[y == 1, 1], 
            c='yellow', s=40, label='1')
plt.scatter(X[y == 2, 0], X[y == 2, 1], 
            c='blue', s=40, label='2')

plt.xlabel("primera característica")
plt.ylabel("segunda característica")
plt.legend(loc='upper right');

Entrenamos el modelo y pintamos sus fronteras de decisión:

In [0]:
import numpy as np

clf = DecisionTreeClassifier(max_features='auto').fit(X, y)

plt.figure(figsize=(10, 5))
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                        np.arange(y_min, y_max, 0.02))
#plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)

Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
cs = plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu)

for i, color in zip(range(3), ('red', 'yellow', 'blue')):
    idx = np.where(y == i)
    plt.scatter(X[idx, 0], X[idx, 1], c=color, label=range(3),
                cmap=plt.cm.RdYlBu, edgecolor='black', s=15)

In [0]:
from sklearn.tree import plot_tree

plot_tree(clf)

Ahora lo único que nos queda por hacer es estudiar el efecto que tienen los hiper-parámetros del árbol de decisión.